# What's new

- structure of stiffness matrix
- stability regions of explicit time integration schemes
- determination and significance of most critical Eigenvalue
- calculation of stable time step $dt_0$
- influence of grid resolution and polynomial degree on time step size


# Prerequisites

- implementation of fluxes, chapter **DifferentialOperator** and **NumFlux**
- explicit time integration, chapter **DifferentialOperator**
- basics of convergence studies, chapters **GridInstantiation** and **NumFlux** 
- grid generation, chapter **GridInstantiation**


In this tutorial, you will learn the basics of time discretization. Using the Upwind, 
Central and two Lax-Friedrichs fluxes the tutorial first visualizes the structure of the stiffness matrix and its Eigenvalues.
Then the stability regions of explicit time integration schemes, e.g. the explicit Euler method, are considered. 
Using the fourth order Runge-Kutta scheme the Eigenvalues are studied for different polynomial orders 
and grid resolutions in order to find the most critical Eigenvalue. Finally, the influence of grid resolution 
and polynomial degree on the time step size is examined and the stable time step $dt_0$ determined in order to obtain a stable and converging solution.

# Problem statement

To keep it simple, the one-dimensional scalar transport equation
$$
\frac{\delta  g}{\delta t} + \nabla \cdot \left(uc\right) = 0
$$
will be used, where $g=g(x,t) \in \mathbb{R}$ is the unknown concentration and $u=1$ the velocity field in a **periodic** domain~$\Omega = \left[-\pi,\pi\right]$.

The linear PDE can be written in its semi-discrete form
$$
\frac{\delta  \vec{g}}{\delta  t} + M \, \vec{g} + **\vec b** = 0
$$
with the unknown DG coefficients~$\vec{g}$, the system or stiffness matrix~$M$ and the affine part~$\vec{b}$. 
The affine part is given by source terms and boundary conditions. In this case, $\vec{b}= 0$.

# Solution within the BoSSS Framework


In [ ]:
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();


# Definition of the operator matrix} \label{sec:scalarConvection_matrix

In [ ]:
using BoSSS.Platform.LinAlg;

In [ ]:
using ilPSP.LinSolvers;

The function~**u** is defined for later usage.

In [ ]:

static Func<double[], double> u = (X => 1.0);

The function~**GetOperatorMatrix** constructs the operator matrix~$M$ corresponding to the semi-discrete ODE system.

In [ ]:

Func<int, int, LinearFlux, MsrMatrix> GetOperatorMatrix =       
    delegate(int numberOfCells, int dgDegree, LinearFlux flux) {       
        double[] nodes = GenericBlas.Linspace(-Math.PI, Math.PI,       
            numberOfCells + 1);       
        GridData gridData = new GridData(Grid1D.LineGrid(       
            nodes, periodic: true));       
 
        Basis basis = new Basis(gridData, dgDegree);       
        SinglePhaseField g = new SinglePhaseField(basis);       
 
        var op = new DifferentialOperator(       
            new string[] { "g" },         // domain variable       
            new string[] { "div" },     // co-domain variable      
            QuadOrderFunc.Linear());       
        op.EquationComponents["div"].Add(flux);       
 
        op.Commit();       
 
        /// The \code{MsrMatrix} is a sparse matrix, where \emph{MSR} stands for \emph{Mutable Sparse Row}. That is, the
        /// matrix can be changed, and the entries are stored in a compressed format where we only have
        /// a small number of column entries per row. This is crucial to be able to handle larger
        /// systems and is required by most linear solvers.
        MsrMatrix operatorMatrix = new MsrMatrix(g.Mapping);       
        double[] affineOffset = new double[g.Mapping.LocalLength];       
        /// Compute the matrix~\code{operatorMatrix} and the affine part~\code{affineOffset} such that an operator
        /// evaluation~$op(g)$ can be expressed as \code{operatorMatrix} $\times$ \code{g} + \code{affineOffset}.
        op.ComputeMatrix(g.Mapping, null, g.Mapping, operatorMatrix,       
            affineOffset, false);       
 
        /// Since we do not enforce any boundary conditions, \code{affineOffset} should be zero.
        if (affineOffset.Any(d => d != 0.0)) {       
            throw new Exception(       
                "We have only periodic boundary conditions."       
                + " Affine part should be zero!");       
        }       
 
        return operatorMatrix;       
    };

# Definition of numerical fluxes

As in the previous tutorials, the Upwind flux is used.

In [ ]:
Func<double, double, double, double, double> upwindFlux =       
    delegate (double Uin, double Uout, double n, double velocity) {       
        if (velocity * n > 0) {       
            return (velocity * Uin) * n;       
        } else {       
            return (velocity * Uout) * n;       
        }       
    };

Moreover, the Lax-Friedrichs flux with a constant **C** (here, different values are used) is implemented.

 If **C** equals zero, the Lax-Friedrichs flux is equivalent to
the central flux.

In [ ]:

Func<double, Func<double, double, double, double, double>>       
  laxFriedrichsFlux =       
    C => delegate (double Uin, double Uout, double n, double velocity) {       
             return 0.5 * (Uin + Uout) * velocity * n - C * (Uout - Uin);       
         };

In general, the flux implementation is similar to the one in the previous tutorials, but in this case, it is one-dimensional and uses **LinearFlux**
as a base class. This class helps to create the entries of the operator matrix discussed above.

In [ ]:

class LinearTransportFlux : LinearFlux {       
 
    private Func<double, double, double, double, double> numericalFlux;       
 
    public LinearTransportFlux(Func<double, double, double, double,       
        double> numericalFlux) {       
            this.numericalFlux = numericalFlux;       
    }       
 
    public override IList<string> ArgumentOrdering {       
        get {       
            return new string[] { "g" };       
        }       
    }       
 
    /// The volume term is very similar to what we have done before.
    protected override void Flux(ref CommonParamsVol inp, double[] U,       
        double[] output) {       
        output[0] = u(inp.Xglobal) * U[0];       
    }       
 
    /// The \code{InnerEdgeFlux} implementation is the same as in the last tutorials, with a slightly different signature.
    protected override double InnerEdgeFlux(ref CommonParams inp,       
        double[] Uin, double[] Uout) {       
            return numericalFlux(Uin[0], Uout[0], inp.Normal[0], u(inp.X));       
    }       
 
    /// We are working on periodic grids only, so the \code{BorderEdgeFlux} is irrelevant.
    protected override double BorderEdgeFlux(ref CommonParamsBnd inp,       
        double[] Uin) {       
            throw new NotImplementedException("Should never be called!");       
    }       
}

# Definition of operators and construction of system matrices

A dictionary is created to access the different fluxes easily.

In [ ]:

var fluxes = new Dictionary<string, Func<double, double, double, double,       
    double>>() {       
        { "Upwind",                   upwindFlux },       
        { "Central",                  laxFriedrichsFlux(0.0) },       
        { "Lax-Friedrichs (C = 0.1)", laxFriedrichsFlux(0.1) },       
        { "Lax-Friedrichs (C = 0.3)", laxFriedrichsFlux(0.3) }       
};

Some initial configurations (polynomial **degree** and **numberOfCells**) are done, as well as defining the system matrices for the above-defined fluxes.


In [ ]:
int degree = 2;       
int numberOfCells = 10;       
MsrMatrix[] matrices = new MsrMatrix[fluxes.Count];       
for (int i = 0; i < fluxes.Count; i++) {       
    LinearFlux flux = new LinearTransportFlux(fluxes.ElementAt(i).Value);       
    matrices[i] = GetOperatorMatrix(numberOfCells, degree, flux);       
}

# Investigation of operator matrices

The style of the plots is defined.

In [ ]:

PlotFormat format = new PlotFormat(       
    Style: Styles.Points,       
    pointType: PointTypes.Asterisk,       
    pointSize: 0.1);

**SetMultiplot** enables multiple plots per graphic.

In [ ]:

Gnuplot gp1 = new Gnuplot(baseLineFormat: format);       
gp1.SetMultiplot(2, fluxes.Count / 2);       
gp1.SetXLabel("Matrix column");       
gp1.SetYLabel("Matrix row");       
// Invert y-axis       
gp1.SetYRange(numberOfCells * (degree + 1), 0);

In [ ]:
for (int i = 0; i < fluxes.Count; i++) {       
    // Select the active sub-plot       
    gp1.SetSubPlot(i / 2, i % 2, title: fluxes.ElementAt(i).Key);       
    /// \code{SetSubPlot} visualizes the non-zero entries of the system matrix. The block size defines the number
    /// of entries per cell. In our case, we have \code{degree} + 1 polynomials per cell such that the
    /// matrix-block associated with a single cell is a (\code{degree} + 1) $\times$ (\code{degree} + 1) sub-matrix.
    gp1.PlotMatrixStructure(matrices[i].ToFullMatrixOnProc0(),       
    blockSize: degree + 1);       
    // Finalize the sub-plot       
    gp1.WriteDeferredPlotCommands();       
}

In [ ]:
gp1.PlotNow()

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 5 
 
 
 
 
 10 
 
 
 
 
 15 
 
 
 
 
 20 
 
 
 
 
 25 
 
 
 
 
 30 
 
 
 
 
 0 
 
 
 
 
 5 
 
 
 
 
 10 
 
 
 
 
 15 
 
 
 
 
 20 
 
 
 
 
 25 
 
 
 
 
 30 
 
 
 
 
 
 
 
 
 Matrix row 
 
 
 
 
 Matrix column 
 
 
 
 
 Upwind 
 
 
 gnuplot_plot_1a 
 
 
 
 
 gnuplot_plot_2a 
 
 
 
 
 gnuplot_plot_3a 
 
 
 
 
 gnuplot_plot_4a 
 
 
 
 
 gnuplot_plot_5a 
 
 
 
 
 gnuplot_plot_6a 
 
 
 
 
 gnuplot_plot_7a 
 
 
 
 
 gnuplot_plot_8a 
 
 
 
 
 gnuplot_plot_9a 
 
 
 
 
 gnuplot_plot_10a 
 
 
 
 
 gnuplot_plot_11a 
 
 
 
 
 gnuplot_plot_12a 
 
 
 
 
 gnuplot_plot_13a 
 
 
 
 
 gnuplot_plot_14a 
 
 
 
 
 gnuplot_plot_15a 
 
 
 
 
 gnuplot_plot_16a 
 
 
 
 
 gnuplot_plot_17a 
 
 
 
 
 gnuplot_plot_18a 
 
 
 
 
 gnuplot_plot_19a 
 
 
 
 
 gnuplot_plot_20a 
 
 
 
 
 gnuplot_plot_21a 
 
 
 
 
 gnuplot_plot_22a 
 
 
 
 
 gnuplot_plot_23a 
 
 
 
 
 gnuplot_plot_24a 
 
 
 
 
 gnuplot_plot_25a 
 
 
 
 
 gnuplot_plot_26a 
 
 
 
 
 gnuplot_plot_27a 
 
 
 
 
 gnuplot_plot_28a 
 
 
 
 
 gnuplot_plot_29a 
 
 
 
 
 gnuplot_plot_30a 
 
 
 
 
 gnuplot_plot_31a 
 
 
 
 
 gnuplot_plot_32a 
 
 
 
 
 gnuplot_plot_33a 
 
 
 
 
 gnuplot_plot_34a 
 
 
 
 
 gnuplot_plot_35a 
 
 
 
 
 gnuplot_plot_36a 
 
 
 
 
 gnuplot_plot_37a 
 
 
 
 
 gnuplot_plot_38a 
 
 
 
 
 gnuplot_plot_39a 
 
 
 
 
 gnuplot_plot_40a 
 
 
 
 
 gnuplot_plot_41a 
 
 
 
 
 gnuplot_plot_42a 
 
 
 
 
 gnuplot_plot_43a 
 
 
 
 
 gnuplot_plot_44a 
 
 
 
 
 gnuplot_plot_45a 
 
 
 
 
 gnuplot_plot_46a 
 
 
 
 
 gnuplot_plot_47a 
 
 
 
 
 gnuplot_plot_48a 
 
 
 
 
 gnuplot_plot_49a 
 
 
 
 
 gnuplot_plot_50a 
 
 
 
 
 gnuplot_plot_51a 
 
 
 
 
 gnuplot_plot_52a 
 
 
 
 
 gnuplot_plot_53a 
 
 
 
 
 gnuplot_plot_54a 
 
 
 
 
 gnuplot_plot_55a 
 
 
 
 
 gnuplot_plot_56a 
 
 
 
 
 gnuplot_plot_57a 
 
 
 
 
 gnuplot_plot_58a 
 
 
 
 
 gnuplot_plot_59a 
 
 
 
 
 gnuplot_plot_60a 
 
 
 
 
 gnuplot_plot_61a 
 
 
 
 
 gnuplot_plot_62a 
 
 
 
 
 gnuplot_plot_63a 
 
 
 
 
 gnuplot_plot_64a 
 
 
 
 
 gnuplot_plot_65a 
 
 
 
 
 gnuplot_plot_66a 
 
 
 
 
 gnuplot_plot_67a 
 
 
 
 
 gnuplot_plot_68a 
 
 
 
 
 gnuplot_plot_69a 
 
 
 
 
 gnuplot_plot_70a 
 
 
 
 
 gnuplot_plot_71a 
 
 
 
 
 gnuplot_plot_72a 
 
 
 
 
 gnuplot_plot_73a 
 
 
 
 
 gnuplot_plot_74a 
 
 
 
 
 gnuplot_plot_75a 
 
 
 
 
 gnuplot_plot_76a 
 
 
 
 
 gnuplot_plot_77a 
 
 
 
 
 gnuplot_plot_78a 
 
 
 
 
 gnuplot_plot_79a 
 
 
 
 
 gnuplot_plot_80a 
 
 
 
 
 gnuplot_plot_81a 
 
 
 
 
 gnuplot_plot_82a 
 
 
 
 
 gnuplot_plot_83a 
 
 
 
 
 gnuplot_plot_84a 
 
 
 
 
 gnuplot_plot_85a 
 
 
 
 
 gnuplot_plot_86a 
 
 
 
 
 gnuplot_plot_87a 
 
 
 
 
 gnuplot_plot_88a 
 
 
 
 
 gnuplot_plot_89a 
 
 
 
 
 gnuplot_plot_90a 
 
 
 
 
 gnuplot_plot_91a 
 
 
 
 
 gnuplot_plot_92a 
 
 
 
 
 gnuplot_plot_93a 
 
 
 
 
 gnuplot_plot_94a 
 
 
 
 
 gnuplot_plot_95a 
 
 
 
 
 gnuplot_plot_96a 
 
 
 
 
 gnuplot_plot_97a 
 
 
 
 
 gnuplot_plot_98a 
 
 
 
 
 gnuplot_plot_99a 
 
 
 
 
 gnuplot_plot_100a 
 
 
 
 
 gnuplot_plot_101a 
 
 
 
 
 gnuplot_plot_102a 
 
 
 
 
 gnuplot_plot_103a 
 
 
 
 
 gnuplot_plot_104a 
 
 
 
 
 gnuplot_plot_105a 
 
 
 
 
 gnuplot_plot_106a 
 
 
 
 
 gnuplot_plot_107a 
 
 
 
 
 gnuplot_plot_108a 
 
 
 
 
 gnuplot_plot_109a 
 
 
 
 
 gnuplot_plot_110a 
 
 
 
 
 gnuplot_plot_111a 
 
 
 
 
 gnuplot_plot_112a 
 
 
 
 
 gnuplot_plot_113a 
 
 
 
 
 gnuplot_plot_114a 
 
 
 
 
 gnuplot_plot_115a 
 
 
 
 
 gnuplot_plot_116a 
 
 
 
 
 gnuplot_plot_117a 
 
 
 
 
 gnuplot_plot_118a 
 
 
 
 
 gnuplot_plot_119a 
 
 
 
 
 gnuplot_plot_120a 
 
 
 
 
 gnuplot_plot_121a 
 
 
 
 
 gnuplot_plot_122a 
 
 
 
 
 gnuplot_plot_123a 
 
 
 
 
 gnuplot_plot_124a 
 
 
 
 
 gnuplot_plot_125a 
 
 
 
 
 gnuplot_plot_126a 
 
 
 
 
 gnuplot_plot_127a 
 
 
 
 
 gnuplot_plot_128a 
 
 
 
 
 gnupl

Now, we can see purple and green blocks in the matrix plots.
The purple blocks are the diagonal blocks that couple the DOF **inside** in a cell.

The off-diagonal green blocks define the coupling between neighboring cells.

Moreover, the differences in the matrix structure for the different fluxes,

especially when comparing the upwind and the central flux to the Lax-Friedrichs variants, is investigate


- **Upwind flux:** Using the upwind flux, the coupling between cells is asymmetric because the flow
         direction is taken into account. As a result, the DOF for a given cell depend on
         the DOF of its left neighbor (green blocks above the diagonal), but not on the DOF of
         its right neighbor (no green blocks below the diagonal in case of Upwind flux).
- **Central flux**: It can be shown (not proven in this tutorial) that the first entry of each
         diagonal block vanishes. That is because of the symmetry of the basis functions, since
         the corresponding term equals zero as well. Please note, that this is a special case for the
         basis function used here.


# Analysis of the spectrum of common explicit ODE integrators

The discrete spectrum of a linear operator in a periodic domain is given by the eigenvalues of
the corresponding system matrix. Here, we will use Matlab to analyze the spectrum
for the different fluxes. For the stability analysis, we have to use the standard form:

$$\frac{\delta \vec u}{\delta t} = -M \, \vec u $$

In [ ]:
using ilPSP.Connectors.Matlab;

The **BatchmodeConnector** initializes an interface to Matlab:

In [ ]:

BatchmodeConnector connector = new BatchmodeConnector();

We will use the array~**eigenValues** to store the eigenvalues of the matrix for each flux. The first index
corresponds to the flux, the second index to the eigenvalue and the third index to the part of
the eigenvalue (0: real part; 1: imaginary part).

In [ ]:

MultidimensionalArray eigenvalues = MultidimensionalArray.Create(       
    fluxes.Count, (int)matrices[0].NoOfCols, 2);

In [ ]:
for (int i = 0; i < fluxes.Count; i++) {       
    // Transfer sparse matrix to Matlab and name the Matlab       
    // variables "M0", "M1", ...       
    connector.PutSparseMatrix(matrices[i], "M" + i);       
    // Compute \emph{all} eigenvalues of the individual matrices       
    // and sort them by magnitude       
    connector.Cmd("eigenvalues{0} = sort(eig(full(M{0})))", i);       
 
    // Separate real and imaginary part; negating the real part       
    // accounts for the minus sign in the standard form discussed above       
    connector.Cmd("result{0} = [-real(eigenvalues{0}) " +       
        "imag(eigenvalues{0})]", i);       
 
    // Retrieve results       
    connector.GetMatrix(eigenvalues.ExtractSubArrayShallow(i, -1, -1),       
        "result" + i);       
}

Calling Matlab takes some time on most machines, even though the individual calculations are
quite fast (suppressing the output). This is due to the slow startup of Matlab...

In [ ]:
connector.Execute(PrintOutput: false);

Now, the eigenvalues are plotted in the complex plane.

In [ ]:
Plot2Ddata[,] specMulPlot = new Plot2Ddata[2,2];

In [ ]:
for (int i = 0; i < fluxes.Count; i++) {       
    string key   = fluxes.ElementAt(i).Key;
    var specPlot = new Plot2Ddata();
    specMulPlot[i / 2, i % 2] = specPlot;
    specPlot.Title     = key;  
    specPlot.XrangeMin = -15;
    specPlot.XrangeMax = +1;
    specPlot.YrangeMin = -15;
    specPlot.YrangeMax = +15;
    specPlot.Xlabel    = "Real";
    specPlot.Ylabel    = "Imaginary";
 
    var p = specPlot.AddDataGroup(
        eigenvalues.ExtractSubArrayShallow(i, -1, 0).To1DArray(),       
        eigenvalues.ExtractSubArrayShallow(i, -1, 1).To1DArray());       
    p.Format.Style     = Styles.Points;
    p.Format.PointSize = 2;
}

In [ ]:
specMulPlot.PlotNow()

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -15 
 
 
 
 
 -10 
 
 
 
 
 -5 
 
 
 
 
 0 
 
 
 
 
 5 
 
 
 
 
 10 
 
 
 
 
 15 
 
 
 
 
 -14 
 
 
 
 
 -12 
 
 
 
 
 -10 
 
 
 
 
 -8 
 
 
 
 
 -6 
 
 
 
 
 -4 
 
 
 
 
 -2 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Imaginary 
 
 
 
 
 Real 
 
 
 
 
 Upwind 
 
 
 gnuplot_plot_1a 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -15 
 
 
 
 
 -10 
 
 
 
 
 -5 
 
 
 
 
 0 
 
 
 
 
 5 
 
 
 
 
 10 
 
 
 
 
 15 
 
 
 
 
 -14 
 
 
 
 
 -12 
 
 
 
 
 -10 
 
 
 
 
 -8 
 
 
 
 
 -6 
 
 
 
 
 -4 
 
 
 
 
 -2 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Imaginary 
 
 
 
 
 Real 
 
 
 
 
 Central 
 
 
 gnuplot_plot_1b 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -15 
 
 
 
 
 -10 
 
 
 
 
 -5 
 
 
 
 
 0 
 
 
 
 
 5 
 
 
 
 
 10 
 
 
 
 
 15 
 
 
 
 
 -14 
 
 
 
 
 -12 
 
 
 
 
 -10 
 
 
 
 
 -8 
 
 
 
 
 -6 
 
 
 
 
 -4 
 
 
 
 
 -2 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Imaginary 
 
 
 
 
 Real 
 
 
 
 
 Lax-Friedrichs (C = 0.1) 
 
 
 gnuplot_plot_1c 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -15 
 
 
 
 
 -10 
 
 
 
 
 -5 
 
 
 
 
 0 
 
 
 
 
 5 
 
 
 
 
 10 
 
 
 
 
 15 
 
 
 
 
 -14 
 
 
 
 
 -12 
 
 
 
 
 -10 
 
 
 
 
 -8 
 
 
 
 
 -6 
 
 
 
 
 -4 
 
 
 
 
 -2 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Imaginary 
 
 
 
 
 Real 
 
 
 
 
 Lax-Friedrichs (C = 0.3) 
 
 
 gnuplot_plot_1d

We will compare the spectra of the different operator matrices.
The spectrum for the central flux case does not contain any negative real parts.
Consequently, the solution of the ODE system is unstable in a sense that
disturbances (e.g. due to round-off or projection errors) will never be damped.
Next, let us have a closer look at the influence of the spectrum of the system matrix
on the maximum admissible time step size. The stability can be analyzed by using the so-called stability function~$f(z)$ of an ODE solver.
That is, the system is stable if all eigenvalues of $dt \, M$ are
located within the stability region $|f(z)| < 1$ of the solver. In the following, we will
nvestigate the stability for the case of the Upwind flux.,

In [ ]:
using System.Numerics;

The stability function of Runge-Kutta schemes for the orders 1 to 4 are stored in a **Dictionary**.

In [ ]:

var stabilityFunctions = new Dictionary<string, Func<Complex, Complex>>() {       
    { "Euler (order 1)",       z => 1 + z },       
    { "Heun (order 2)",        z => 1.0 + z + 0.5*z*z },       
    { "Kutta (order 3)",       z => 1.0 + z + 0.5*z*z + 1.0/6.0*z*z*z },       
    { "Runge-Kutta (order 4)", z => 1.0 + z + 0.5*z*z + 1.0/6.0*z*z*z +        
        1.0/24.0*z*z*z*z }       
};

We create some sampling nodes for the following plots.

In [ ]:

double[] xNodes = GenericBlas.Linspace(-4.0, 1.0, 50).ToArray();       
double[] yNodes = GenericBlas.Linspace(-4.0, 4.0, 100).ToArray();

We plot the boundary of the stability regions for the different schemes and orders, respectively.

In [ ]:

Gnuplot gp3 = new Gnuplot(baseLineFormat: format);       
gp3.SetMultiplot(2, stabilityFunctions.Count / 2);       
gp3.SetXLabel("Real");       
gp3.SetYLabel("Imaginary");       
gp3.Cmd("set grid");       
gp3.Cmd("set size square");       
gp3.SetXRange(-4.0, 4.0);       
gp3.SetYRange(-4.0, 4.0);

In [ ]:
for (int i = 0; i < stabilityFunctions.Count; i++) {       
    string key = stabilityFunctions.ElementAt(i).Key;       
    gp3.SetSubPlot(i / 2, i % 2, title: key);       
 
    /// We plot the iso-contour where $|\code{stabilityFunction}| = 1.0$.
    gp3.PlotContour(       
        xNodes,       
        yNodes,       
        (x, y) => stabilityFunctions[key](new Complex(x, y)).Magnitude,       
        new double[] { 1.0 },       
        title: key);       
 
    gp3.WriteDeferredPlotCommands();       
}

In [ ]:
gp3.PlotNow()

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Euler (order 1) 
 
 
 gnuplot_plot_1a 
 
 
 
 
 
	<path stroke='rgb( 0, 158, 115)' d='M255.7,195.6 L255.4,197.9 L255.3,197.9 L254.8,200.3 L254.0,202.6 L252.9,205.0 L252.4,206.0 L251.6,207.3
 L250.0,209.7 L249.4,210.4 L248.0,212.0 L246.4,213.6 L245.5,214.4 L243.5,215.9 L242.2,216.7 L240.5,217.7
 L237.6,219.1 L237.5,219.1 L234.6,220.1 L231.6,220.8 L228.6,221.1 L225.7,221.1 L222.7,220.9 L219.7,220.3
 L216.7,219.3 L216.2,219.1 L213.8,218.1 L211.5,216.7 L210.8,216.3 L208.2,214.4 L207.8,214.1 L205.7,212.0
 L204.9,211.1 L203.7,209.7 L202.1,207.3 L201.9,207.0 L200.8,205.0 L199.7,202.6 L198.9,200.3 L198.3,197.9
 L198.0,195.6 L197.8,193.2 L197.8,191.0 L198.0,188.6 L198.3,186.3 L198.9,183.9 L199.7,181.6 L200.8,179.2
 L201.9,177.2 L202.1,176.9 L203.7,174.5 L204.9,173.1 L205.7,172.2 L207.8,170.1 L208.2,169.8 L210.8,167.9
 L211.5,167.5 L213.8,166.1 L216.2,165.1 L216.7,164.9 L219.7,163.9 L222.7,163.3 L225.7,163.1 L228.6,163.1
 L231.6,163.4 L234.6,164.1 L237.5,165.1 L237.6,165.1 L240.5,166.5 L242.2,167.5 L243.5,168.3 L245.5,169.8
 L246.4,170.6 L248.0,172.2 L249.4,173.8 L250.0,174.5 L251.6,176.9 L252.4,178.2 L252.9,179.2 L254.0,181.6
 L254.8,183.9 L255.3,186.3 L255.4,186.3 L255.7,188.6 L255.9,191.0 L255.9,193.2 L255.7,195.6 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 4 
 
 
 
 
 Real 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 4 
 
 
 
 
 Imaginary 
 
 
 
 
 
 
 Heun (order 2) 
 
 
 gnuplot_plot_1b 
 
 
 
 
 
	<path stroke='rgb( 0, 158, 115)' d='M761.4,154.7 L762.1,155.7 L763.3,158.0 L764.4,160.4 L765.2,162.8 L765.9,165.1 L766.5,167.5 L766.9,169.8
 L767.3,172.2 L767.4,172.9 L767.5,174.5 L767.7,176.9 L767.8,179.2 L767.9,181.6 L767.9,183.9 L767.9,186.3
 L767.9,188.6 L767.9,191.0 L767.9,193.2 L767.9,195.6 L767.9,197.9 L767.9,200.3 L767.9,202.6 L767.8,205.0
 L767.7,207.3 L767.5,209.7 L767.4,211.3 L767.3,212.0 L766.9,214.4 L766.5,216.7 L765.9,219.1 L765.2,221.4
 L764.4,223.8 L763.3,226.2 L762.1,228.5 L761.4,229.5 L760.5,230.9 L758.7,233.2 L758.4,233.5 L756.3,235.6
 L755.5,236.3 L753.3,237.9 L752.5,238.5 L749.5,240.1 L749.1,240.3 L746.6,241.2 L743.6,242.0 L740.6,242.4
 L737.7,242.4 L734.7,242.1 L731.7,241.4 L728.7,240.4 L728.5,240.3 L725.8,238.8 L724.4,237.9 L722.8,236.8
 L721.4,235.6 L719.8,234.1 L719.0,233.2 L717.2,230.9 L716.9,230.4 L715.6,228.5 L714.4,226.2 L713.9,225.1
 L713.3,223.8 L712.5,221.4 L711.8,219.1 L711.2,216.7 L710.9,215.3 L710.8,214.4 L710.4,212.0 L710.2,209.7
 L710.0,207.3 L709.9,205.0 L709.8,202.6 L709.8,200.3 L709.8,197.9 L709.8,195.6 L709.8,193.2 L709.8,191.0
 L709.8,188.6 L709.8,186.3 L709.8,183.9 L709.8,181.6 L709.9,179.2 L710.0,176.9 L710.2,174.5 L710.4,172.2
 L710.8,169.8 L710.9,168.9 L711.2,167.5 L711.8,165.1 L712.5,162.8 L713.3,160.4 L713.9,159.1 L714.4,158.0
 L715.6,155.7 L716.9,153.8 L717.2,153.3 L719.0,151.0 L719.8,150.1 L721.4,148.6 L722.8,147.4 L724.4,146.3
 L725.8,145.4 L728.5,143.9 L728.7,143.8 L731.7,142.8 L734.7,142.1 L737.7,141.8 L740.6,141.8 L743.6,142.2
 L746.6,143.0 L749.1,143.9 L749.5,144.1 L752.5,145.7 L753.3,146.3 L755.5,147.9 L756.3,148.6 L758.4,150.7
 L758.7,151.0 L760.5,153.3 L761.4,154.7 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

# Stability region of the fourth order Runge-Kutta scheme

 We will now focus on the fourth order Runge-Kutta scheme.
  For some obscure reasons, **gnuplot** forces us to write the contour plot data from above into a file in order to
  be able to use it in normal plots.
  
   The syntax is completely irrelevant for our
  purposes, so please do not waste time on trying to understand it.

In [ ]:
string dataFile = "rk4.dat";       
Gnuplot gp4 = new Gnuplot();       
gp4.Cmd("set table '{0}'", dataFile);       
gp4.PlotContour(       
    xNodes,       
    yNodes,       
    (x, y) => stabilityFunctions["Runge-Kutta (order 4)"](       
        new Complex(x, y)).Magnitude, new double[] { 1.0 });       
gp4.Cmd("unset table");       
gp4.PlotNow()

<null>

Give gnuplot some time to write the file before using it in the next plot.    

In [ ]:
  
System.Threading.Thread.Sleep(2000);

The maximum stable time step, such that the solution is stable for the fourth order Runge-Kutta method is **dtMax=0.14**, see next plot.

In [ ]:
double dtMax = 0.14;

In [ ]:
Gnuplot gp5 = new Gnuplot(baseLineFormat: format);       
gp5.SetXLabel("Real");       
gp5.SetYLabel("Imaginary");       
var realPart = eigenvalues.ExtractSubArrayShallow(0, -1, 0).To1DArray()       
    .Select(x => dtMax * x);       
var imaginaryPart = eigenvalues.ExtractSubArrayShallow(0, -1, 1).To1DArray()       
    .Select(y => dtMax * y);       
gp5.PlotXY(       
    realPart,       
    imaginaryPart,       
    title: "dt = " + dtMax);       
gp5.PlotDataFile(dataFile, title: "Runge-Kutta", format: new PlotFormat(       
    Style: Styles.Lines));       
gp5.PlotNow()

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -3 
 
 
 
 
 -2 
 
 
 
 
 -1 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 -3 
 
 
 
 
 -2.5 
 
 
 
 
 -2 
 
 
 
 
 -1.5 
 
 
 
 
 -1 
 
 
 
 
 -0.5 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 
 
 
 
 Imaginary 
 
 
 
 
 Real 
 
 
 
 
 dt = 0.14 
 
 
 dt = 0.14 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Runge-Kutta 
 
 
 Runge-Kutta 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M940.2,52.7 L982.4,52.7 M890.3,555.1 L896.2,564.4 L902.6,573.8 L909.2,583.1 L913.7,589.5 L915.3,592.4
 L920.2,601.8 L924.2,611.1 L926.8,620.5 L927.8,629.8 L926.9,639.2 L923.7,648.5 L918.0,657.8 M280.9,162.9
 L281.6,162.7 L308.4,153.9 L310.3,153.3 L335.9,145.9 L343.3,144.0 L363.4,138.7 L379.9,134.7 L391.0,131.9
 L418.4,125.3 L418.5,125.3 L446.0,118.8 L457.3,116.0 L473.5,111.8 L492.1,106.6 L501.0,104.0 L521.9,97.3
 L528.5,95.1 L547.5,87.9 L556.0,84.7 L570.1,78.6 L583.6,72.9 L591.3,69.3 L611.1,60.7 L612.6,59.9
 L635.5,50.6 L638.6,49.5 L662.5,41.2 L666.1,40.2 L693.6,33.0 L699.0,31.9 L721.1,28.1 L748.6,25.0
 L776.1,24.0 L803.7,25.2 L831.2,28.7 L846.2,31.9 L858.7,35.0 L876.5,41.2 L886.2,45.5 L895.7,50.6
 L908.9,59.9 L913.7,64.7 L918.0,69.3 L923.7,78.6 L926.9,87.9 L927.8,97.3 L926.8,106.6 L924.2,116.0
 L920.2,125.3 L915.3,134.7 L913.7,137.6 L909.2,144.0 L902.6,153.3 L896.2,162.7 L890.3,172.0 L886.2,179.5
 L885.0,181.4 L879.8,190.7 L875.6,200.1 L872.4,209.4 L869.9,218.7 L868.0,228.1 L866.6,237.4 L865.7,246.8
 L865.0,256.1 L864.6,265.5 L864.3,274.8 L864.1,284.1 L864.0,293.5 L864.0,302.8 L864.0,312.2 L864.0,321.5
 L864.0,330.9 L864.0,340.2 L864.0,349.5 L864.0,358.9 L864.0,368.2 L864.0,377.6 L864.0,386.9 L864.0,396.2
 L864.0,405.6 L864.0,414.9 L864.0,424.3 L864.0,433.6 L864.1,443.0 L864.3,452.3 L864.6,461.6 L865.0,471.0
 L865.7,480.3 L866.6,489.7 L868.0,499.0 L869.9,508.4 L872.4,517.7 L875.6,527.0 L879.8,536.4 L885.0,545.7
 L886.2,547.6 L890.3,555.1 M918.0,657.8 L913.7,662.4 L908.9,667.2 L895.7,676.5 L886.2,681.6 L876.5,685.9
 L858.7,692.1 L846.2,695.2 L831.2,698.4 L803.7,701.9 L776.1,703.1 L748.6,702.1 L721.1,699.0 L699.0,695.2
 L693.6,694.1 L666.1,686.9 L662.5,685.9 L638.6,677.6 L635.5,676.5 L612.6,667.2 L611.1,666.4 L591.3,657.8
 L583.6,654.2 L570.1,648.5 L556.0,642.4 L547.5,639.2 L528.5,632.0 L521.9,629.8 L501.0,623.1 L492.1,620.5
 L473.5,615.3 L457.3,611.1 L446.0,608.3 L418.5,601.8 L418.4,601.8 L391.0,595.2 L379.9,592.4 L363.4,588.4
 L343.3,583.1 L335.9,581.2 L310.3,573.8 L308.4,573.2 L281.6,564.4 L280.9,564.2 L257.0,555.1 L253.4,553.7
 L235.6,545.7 L225.9,541.2 L216.7,536.4 L200.0,527.0 L198.4,526.0 L186.1,517.7 L173.5,508.4 L170.9,506.2
 L162.9,499.0 L153.7,489.7 L145.6,480.3 L143.3,477.4 L139.0,471.0 L133.5,461.6 L128.8,452.3 L124.9,443.0
 L121.7,433.6 L119.2,424.3 L117.2,414.9 L115.8,406.8 L115.6,405.6 L114.7,396.2 L114.0,386.9 L113.6,377.6
 L113.4,368.2 L113.4,358.9 L113.6,349.5 L114.0,340.2 L114.7,330.9 L115.6,321.5 L115.8,320.3 L117.2,312.2
 L119.2,302.8 L121.7,293.5 L124.9,284.1 L128.8,274.8 L133.5,265.5 L139.0,256.1 L143.3,249.7 L145.6,246.8
 L153.7,237.4 L162.9,228.1 L170.9,220.9 L173.5,218.7 L186.1,209.4 L198.4,201.1 L200.0,200.1 L216.7,190.7
 L225.9,185.9 L235.6,181.4 L253.4,173.4 L257.0,172.0 L280.9,162.9 '/>

# Advanced topics
## Scaling of the step-size w.r.t. grid size and polynomial degree

Having found a stable step size for some configuration, we will now study the question how
the maximum admissible time step size changes when we increase the number of cells and/or increase
the ansatz order. We will study the following combinations of grid size and polynomial degree.

In [ ]:
int[] numbersOfCells = new int[] { 4, 8, 16, 32 };       
int[] orders = new int[] { 0, 1, 2, 3, 4 };

Therefore, we have again to set up a **BatchmodeConnector** to Matlab for calculating the eigenvalues.

In [ ]:

BatchmodeConnector connector = new BatchmodeConnector();       
MultidimensionalArray[,] eigenvalueStudy = new MultidimensionalArray[       
    orders.Length, numbersOfCells.Length];       
for (int j = 0; j < orders.Length; j++) {       
    for (int i = 0; i < numbersOfCells.Length; i++) {       
        var matrix = GetOperatorMatrix(       
            numbersOfCells[i],       
            orders[j],       
            new LinearTransportFlux(upwindFlux));       
 
        connector.PutSparseMatrix(matrix, "M" + i + j);       
        connector.Cmd("eigenvalues{0}{1} = eig(full(M{0}{1}))", i, j);       
        connector.Cmd("result{0}{1} = [-real(eigenvalues{0}{1}) " +       
           "imag(eigenvalues{0}{1})]", i, j);       
 
        eigenvalueStudy[j, i] = MultidimensionalArray.Create(       
           (int)matrix.NoOfCols, 2);       
        connector.GetMatrix(eigenvalueStudy[j, i], "result" + i + j);       
    }       
}       
connector.Execute(false);

The time step size~**dt** is selected arbitrarily to get instructive figures.

In [ ]:

double dt = 0.3;       
int numberOfPlotRowsAndCols = 3;

Now, the spectrum of the system matrices is plotted for 4, 8 and 16 cells for the
orders 0, 1 and 2. You can clearly see that for an increasing order all of the spectra
mostly grow in the negative direction. That is why the most negative eigenvalue is
considered as the most critical one in terms of the time step restriction.

In [ ]:

Gnuplot gp6 = new Gnuplot(baseLineFormat: format);       
gp6.SetMultiplot(numberOfPlotRowsAndCols, numberOfPlotRowsAndCols);       
gp6.SetXRange(-10.0, 1.0);       
gp6.SetYRange(-5.0, 5.0);

In [ ]:
for (int i = 0; i < numberOfPlotRowsAndCols; i++) {       
    for (int j = 0; j < numberOfPlotRowsAndCols; j++) {       
        gp6.SetSubPlot(j, i);       
        gp6.SetTitle(String.Format("{0} cells, order {1}", numbersOfCells[i],       
           orders[j]));       
        gp6.PlotXY(       
            eigenvalueStudy[j, i].ExtractSubArrayShallow(-1, 0).To1DArray()       
               .Select(re => dt * re),       
            eigenvalueStudy[j, i].ExtractSubArrayShallow(-1, 1).To1DArray()       
               .Select(im => dt * im));       
        gp6.PlotDataFile(dataFile, format: new PlotFormat(       
           Style: Styles.Lines));       
        gp6.WriteDeferredPlotCommands();       
    }       
}

In [ ]:
gp6.PlotNow()

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -4 
 
 
 
 
 -2 
 
 
 
 
 0 
 
 
 
 
 2 
 
 
 
 
 4 
 
 
 
 
 -10 
 
 
 
 
 -8 
 
 
 
 
 -6 
 
 
 
 
 -4 
 
 
 
 
 -2 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 4 cells, order 0 
 
 
 gnuplot_plot_1a 
 
 
 
 
 
 
 
 gnuplot_plot_2a 
 
	<path stroke='rgb(255, 0, 0)' d='M290.6,163.7 L291.1,165.0 L291.7,166.3 L292.3,167.6 L292.7,168.5 L292.8,168.9 L293.3,170.2 L293.7,171.5
 L293.9,172.8 L294.0,174.1 L293.9,175.4 L293.6,176.7 L293.1,178.0 M234.6,109.2 L234.7,109.2 L237.1,107.9
 L237.3,107.9 L239.7,106.8 L240.4,106.6 L242.2,105.8 L243.7,105.3 L244.7,104.9 L247.2,104.0 L249.8,103.1
 L250.8,102.7 L252.3,102.1 L254.0,101.4 L254.8,101.0 L256.7,100.1 L257.3,99.8 L259.1,98.8 L259.9,98.3
 L261.2,97.5 L262.4,96.7 L263.1,96.2 L264.9,95.0 L265.1,94.9 L267.2,93.6 L267.4,93.4 L269.6,92.3
 L270.0,92.1 L272.5,91.1 L273.0,91.0 L275.0,90.4 L277.5,90.0 L280.1,89.9 L282.6,90.0 L285.1,90.5
 L286.5,91.0 L287.6,91.4 L289.3,92.3 L290.2,92.9 L291.0,93.6 L292.3,94.9 L292.7,95.5 L293.1,96.2
 L293.6,97.5 L293.9,98.8 L294.0,100.1 L293.9,101.4 L293.7,102.7 L293.3,104.0 L292.8,105.3 L292.7,105.7
 L292.3,106.6 L291.7,107.9 L291.1,109.2 L290.6,110.5 L290.2,111.5 L290.1,111.8 L289.6,113.1 L289.2,114.4
 L288.9,115.7 L288.7,117.0 L288.5,118.3 L288.4,119.6 L288.3,120.9 L288.2,122.2 L288.2,123.5 L288.2,124.8
 L288.1,126.1 L288.1,127.4 L288.1,128.7 L288.1,130.0 L288.1,131.3 L288.1,132.6 L288.1,133.9 L288.1,135.2
 L288.1,136.5 L288.1,137.7 L288.1,139.0 L288.1,140.3 L288.1,141.6 L288.1,142.9 L288.1,144.2 L288.1,145.5
 L288.1,146.8 L288.1,148.1 L288.2,149.4 L288.2,150.7 L288.2,152.0 L288.3,153.3 L288.4,154.6 L288.5,155.9
 L288.7,157.2 L288.9,158.5 L289.2,159.8 L289.6,161.1 L290.1,162.4 L290.2,162.7 L290.6,163.7 M293.1,178.0
 L292.7,178.7 L292.3,179.3 L291.0,180.6 L290.2,181.3 L289.3,181.9 L287.6,182.8 L286.5,183.2 L285.1,183.7
 L282.6,184.2 L280.1,184.3 L277.5,184.2 L275.0,183.8 L273.0,183.2 L272.5,183.1 L270.0,182.1 L269.6,181.9
 L267.4,180.8 L267.2,180.6 L265.1,179.3 L264.9,179.2 L263.1,178.0 L262.4,177.5 L261.2,176.7 L259.9,175.9
 L259.1,175.4 L257.3,174.4 L256.7,174.1 L254.8,173.2 L254.0,172.8 L252.3,172.1 L250.8,171.5 L249.8,171.1
 L247.2,170.2 L244.7,169.3 L243.7,168.9 L242.2,168.4 L240.4,167.6 L239.7,167.4 L237.3,166.3 L237.1,166.3
 L234.7,165.0 L234.6,165.0 L232.4,163.7 L232.1,163.5 L230.5,162.4 L229.6,161.8 L228.7,161.1 L227.2,159.8
 L227.0,159.7 L225.9,158.5 L224.8,157.2 L224.5,156.9 L223.8,155.9 L223.0,154.6 L222.2,153.3 L222.0,152.9
 L221.6,152.0 L221.1,150.7 L220.7,149.4 L220.3,148.1 L220.0,146.8 L219.8,145.5 L219.6,144.2 L219.5,143.1
 L219.5,142.9 L219.4,141.6 L219.3,140.3 L219.3,139.0 L219.2,137.7 L219.2,136.5 L219.3,135.2 L219.3,133.9
 L219.4,132.6 L219.5,131.3 L219.5,131.1 L219.6,130.0 L219.8,128.7 L220.0,127.4 L220.3,126.1 L220.7,124.8
 L221.1,123.5 L221.6,122.2 L222.0,121.3 L222.2,120.9 L223.0,119.6 L223.8,118.3 L224.5,117.3 L224.8,117.0
 L225.9,115.7 L227.0,114.5 L227.2,114.4 L228.7,113.1 L229.6,112.4 L230.5,111.8 L232.1,110.7 L232.4,110.5
 L234.6,109.2 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -4 
 
 
 
 
 -2 
 
 
 
 
 0 
 
 
 
 
 2 
 
 
 
 
 4 
 
 
 
 
 -10 
 
 
 
 
 -8 
 
 
 
 
 -6 
 
 
 
 
 -4 
 
 
 
 
 -2 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 4 cells, order 1 
 
 
 gnuplot_plot_1b 
 
 
 
 
 
 
 
 
 
 
 
 gnuplot_plot_2b 
 
	<path stroke='rgb(255, 0, 0)' d='M290.6,419.7 L291.1,421.0 L291.7,422.3 L292.3,423.6 L292.7,424.5 L292.8,424.9 L293.3,426.2 L293.7,427.5
 L293.9,428.8 L294.0,430.1 L293.9,431.4 L293.6,432.7 L293.1,434.0 M234.6,365.1 L234.7,365.1 L237.1,363.9
 L237.3,363.8 L239.7,362.8 L240.4,362.5 L242.2,361.8 L243.7,361.2 L244.7,360.8 L247.2,359.9 L249.8,359.0
 L250.8,358.6 L252.3,358.0 L254.0,357.3 L254.8,356.9 L256.7,356.0 L257.3,355.7 L259.1,354.7 L259.9,354.2
 L261.2,353.4 L262.4,352.6 L263.1,352.1 

In [ ]:
MultidimensionalArray criticalValues = MultidimensionalArray.Create(       
   orders.Length, numbersOfCells.Length);

For each combination of cells and polynomial order, the magnitude of the
critical eigenvalue identified above is stored in the respective entry of **criticalValues**.

In [ ]:
for (int j = 0; j < orders.Length; j++) {       
    for (int i = 0; i < numbersOfCells.Length; i++) {       
        criticalValues[j, i] = -eigenvalueStudy[j, i].       
           ExtractSubArrayShallow(-1, 0).To1DArray().Min();       
    }       
}

The mesh size is stored in the array~**oneOverMeshSize**.

In [ ]:

double[] oneOverMeshSize = numbersOfCells.Select<int, double>(       
 
   delegate (int n){return n / 2.0 * Math.PI;}).ToArray();

The polynomial order is stored in the array~**numberOfBasisPolynomials**. 

In [ ]:

double[] numberOfBasisPolynomials = orders.Select(o => o + 1.0).ToArray();

A reference value is set for normalization purposes. It defines the maximum stable step-size on the coarsest
grid using a zeroth order approximation.

In [ ]:

double minCriticalValue = criticalValues[0, 0];

In the following, the decrease of the stable step-size is plotted when varying the grid resolution for a
given ansatz order (assuming that the step-size scales with 1/**criticalValues[j, i]**).

In [ ]:
PlotFormat format2 = new PlotFormat(       
    Style: Styles.LinesPoints,       
    pointType: PointTypes.Asterisk,       
    pointSize: 1.0);

In [ ]:
Gnuplot gp7 = new Gnuplot(baseLineFormat: format2);       
gp7.SetTitle("Relative maximum time-step size for various orders");       
gp7.SetXLabel("Log(1/h)");       
gp7.SetYLabel("Log(Relative step-size)");

In [ ]:
for (int j = 0; j < orders.Length; j++) {       
    var stepSize = criticalValues.ExtractSubArrayShallow(j, -1).To1DArray()       
       .Select(d => minCriticalValue / d);       
    gp7.PlotLogXLogY(       
        oneOverMeshSize,       
        stepSize,       
        title: "Order " + orders[j],       
        format: new PlotFormat(Style: Styles.LinesPoints,       
                               pointType: PointTypes.Asterisk,       
                               pointSize: 1.0,       
                               lineColor: (LineColors)(j+1))       
        );       
}

In [ ]:
gp7.PlotNow()

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.001 
 
 
 
 
 0.01 
 
 
 
 
 0.1 
 
 
 
 
 1 
 
 
 
 
 1 
 
 
 
 
 10 
 
 
 
 
 100 
 
 
 
 
 
 
 
 
 Log(Relative step-size) 
 
 
 
 
 Log(1/h) 
 
 
 
 
 Relative maximum time-step size for various orders 
 
 
 Order 0 
 
 
 Order 0 
 
 
 
 
 
 
 
 
 
 
 Order 1 
 
 
 Order 1 
 
 
 
 
 
 
 
 
 
 
 Order 2 
 
 
 Order 2 
 
 
 
 
 
 
 
 
 
 
 Order 3 
 
 
 Order 3 
 
 
 
 
 
 
 
 
 
 
 Order 4 
 
 
 Order 4

In contrast to what we have done before, the decrease of the stable step-size is 
plotted when varying the ansatz order on a given 
grid (assuming that the step-size scales with **1/[criticalValues[j, i]**

In [ ]:
Gnuplot gp8 = new Gnuplot(baseLineFormat: format2);       
gp8.SetTitle("Relative maximum time-step size for various grid sizes");       
gp8.SetXLabel("Log(Number of basis polynomials)");       
gp8.SetYLabel("Log(Relative step-size)");

In [ ]:
for (int i = 0; i < numbersOfCells.Length; i++) {       
    var stepSize = criticalValues.ExtractSubArrayShallow(-1, i).To1DArray()       
       .Select(d => minCriticalValue / d);       
    gp8.PlotLogXLogY(       
        numberOfBasisPolynomials,       
        stepSize,       
        title: numbersOfCells[i] + " cells per direction",       
        format: new PlotFormat(Style: Styles.LinesPoints,       
                               pointType: PointTypes.Asterisk,       
                               pointSize: 1.0,       
                               lineColor: (LineColors)(i+1))       
    );       
}

In [ ]:
gp8.PlotNow()

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.001 
 
 
 
 
 0.01 
 
 
 
 
 0.1 
 
 
 
 
 1 
 
 
 
 
 1 
 
 
 
 
 10 
 
 
 
 
 
 
 
 
 Log(Relative step-size) 
 
 
 
 
 Log(Number of basis polynomials) 
 
 
 
 
 Relative maximum time-step size for various grid sizes 
 
 
 4 cells per direction 
 
 
 4 cells per direction 
 
 
 
 
 
 
 
 
 
 
 
 8 cells per direction 
 
 
 8 cells per direction 
 
 
 
 
 
 
 
 
 
 
 
 16 cells per direction 
 
 
 16 cells per direction 
 
 
 
 
 
 
 
 
 
 
 
 32 cells per direction 
 
 
 32 cells per direction

Having a look at the two previously created plots suggests to define
a scaling law for the estimation of stable time step sizes depending on h and n.
The stable time step size~$dt_0$ for a zeroth order DG approximation ($p=0\Rightarrow n=1$)
is given on a mesh with the grid size~$h$. To do so, we want to use this ansatz for the scaling law
$$
dt = dt_0 \frac{h^a}{n^b}
$$
by reusing the method~**slope** from tutorial 5 to find a and b.

In [ ]:
Func<double[], double[], double> slope = delegate(double[] xValues,       
   double[] yValues) {       
    if (xValues.Length != yValues.Length) {       
        throw new ArgumentException();       
    }       
 
    xValues = xValues.Select(s => Math.Log10(s)).ToArray();       
    yValues = yValues.Select(s => Math.Log10(s)).ToArray();       
 
    double xAverage = xValues.Sum() / xValues.Length;       
    double yAverage = yValues.Sum() / yValues.Length;       
 
    double v1 = 0.0;       
    double v2 = 0.0;       
 
    for (int i = 0; i < yValues.Length; i++) {       
        v1 += (xValues[i] - xAverage) * (yValues[i] - yAverage);       
        v2 += Math.Pow(xValues[i] - xAverage, 2);       
    }       
 
    return v1 / v2;       
};

In [ ]:
using NUnit.Framework;

In [ ]:
var stepSize = criticalValues.ExtractSubArrayShallow(0, -1).To1DArray();       
slope(oneOverMeshSize, stepSize);

In [ ]:
Assert.LessOrEqual(Math.Abs(slope(oneOverMeshSize, stepSize)-1),1E-01);

In [ ]:
var stepSize = criticalValues.ExtractSubArrayShallow(-1, 0).To1DArray();       
slope(numberOfBasisPolynomials, stepSize);

In [ ]:
Assert.LessOrEqual(Math.Abs(slope(numberOfBasisPolynomials, stepSize)-1.6),1E-01);

The resulting scaling law yields:
$$
dt \propto \frac{h^1}{n^{1.63}}
$$